<a href="https://colab.research.google.com/github/BennettHilck12/DraftEdge/blob/main/DraftEdge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS 5450 Final Project: DraftEdge

##### Bennett Hilck, Ethan Xia, Mohammed Soufan


# Part 1: Introduction



For our final project, we sought to better understand how NBA teams can draft players more effectively by predicting which NCAA D1 prospects are most likely to outperform expectations at the professional level. To pursue this goal, we compiled and analyzed several datasets containing college performance statistics, NBA Combine measurements and performance statistics, and eventual NBA outcomes.


By aggregrating and merging these data sources, we aim to identify advanced or undervalued metrics that may be overlook in traditional scouting evaluations. This approach alows us to quantitatvely asses why certain highly drafted players become "busts," while others selected later dramatically exceed their draft position.


Given the high variance and high cost associated with drafting, our project highlights non-obvious predictors that could help inform more data-driven decision-making and potentially reshape how teams and scouts approach future NBA drafts.


We hope that you find our project impactful!

# Part 2: Data Loading & Preprocessing


First, we must import all relevant libraries for our project. We will use all supplemental libraries throughout our project.

In [ ]:
# Imports + Installs
!pip install category_encoders
import pandas as pd
import seaborn as sns
import dask.dataframe as dd
import matplotlib.pyplot as plt
import folium
import numpy as np
import category_encoders as ce
import dask.array as da
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tqdm
import copy
from xgboost import XGBRegressor
from folium.plugins import HeatMap
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import spearmanr
from sklearn import datasets, linear_model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from torch.utils import data as data_utils
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

## 2.1 Data Mounting + Storage

Because our datasets are quite large, often containing thousands of player seasons, detailed college statistics, and multiple years of NBA Combine measurements—we decided to upload them to Google Drive rather than store them locally. This allows us to efficiently access and manage the data from within our notebook environment without running into storage limitations or upload constraints. Hosting the data on Google Drive also ensures easier collaboration, as all team members can work from the same centralized, version-consistent files.

In [ ]:
# Data Loading via Google Docs
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2.2 Loading and Preprocessing NBA Combine Data

First, let's load in our data from the NBA combine.

In [ ]:
file_path = '/content/drive/MyDrive/final_project_datasets/draft_combine_stats.csv'
combine_df = pd.read_csv(file_path)
display(combine_df.head(10))

,season,player_id,first_name,last_name,player_name,position,height_wo_shoes,height_wo_shoes_ft_in,height_w_shoes,height_w_shoes_ft_in,...,spot_nba_break_right,spot_nba_corner_right,off_drib_fifteen_break_left,off_drib_fifteen_top_key,off_drib_fifteen_break_right,off_drib_college_break_left,off_drib_college_top_key,off_drib_college_break_right,on_move_fifteen,on_move_college
0,2001,12033,Adam,Allenspach,Adam Allenspach,C,83.50,6' 11.5'',NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,2240,Gilbert,Arenas,Gilbert Arenas,SG,74.25,6' 2.25'',NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2001,2220,Brandon,Armstrong,Brandon Armstrong,SG,75.50,6' 3.5'',NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001,2203,Shane,Battier,Shane Battier,SF-PF,80.25,6' 8.25'',NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001,12034,Cookie,Belcher,Cookie Belcher,SG-PG,75.00,6' 3'',NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2001,2294,Charlie,Bell,Charlie Bell,PG,74.50,6' 2.5'',NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2001,2257,Ruben,Boumtje-Boumtje,Ruben Boumtje-Boumtje,C,83.50,6' 11.5'',NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2001,12035,Calvin,Bowman,Calvin Bowman,PF,80.75,6' 8.75'',NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2001,2214,Michael,Bradley,Michael Bradley,PF,81.50,6' 9.5'',NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2001,2249,Jamison,Brewer,Jamison Brewer,PG,74.50,6' 2.5'',NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
combine_df.dtypes

,0
season,int64
player_id,int64
first_name,object
last_name,object
player_name,object
position,object
height_wo_shoes,float64
height_wo_shoes_ft_in,object
height_w_shoes,float64
height_w_shoes_ft_in,object


In [ ]:
combine_df.describe()

,season,player_id,height_wo_shoes,height_w_shoes,weight,wingspan,standing_reach,body_fat_pct,hand_length,hand_width,standing_vertical_leap,max_vertical_leap,lane_agility_time,modified_lane_agility_time,three_quarter_sprint,bench_press
count,1202.000000,1.202000e+03,1153.000000,1008.000000,1152.000000,1153.000000,1152.000000,1003.000000,719.000000,719.000000,1017.000000,1017.000000,1008.000000,411.000000,1012.000000,808.000000
mean,2012.536606,2.350133e+06,77.570902,78.796577,214.902604,82.478187,103.593663,7.352313,8.720793,9.448887,29.240167,34.637168,11.384444,3.079221,3.282213,10.155941
std,6.561620,5.660210e+07,3.334605,3.322625,25.718878,3.986623,4.880218,2.748712,0.481028,0.717066,3.054645,3.648106,0.584896,0.229143,0.131850,5.389231
min,2001.000000,-1.000000e+00,67.750000,69.000000,154.400000,70.000000,89.500000,2.600000,7.500000,7.000000,20.500000,25.000000,9.650000,2.220000,2.910000,0.000000
25%,2007.000000,1.011455e+05,75.250000,76.500000,195.950000,80.000000,100.000000,5.400000,8.500000,9.000000,27.000000,32.000000,10.970000,2.970000,3.190000,6.000000
50%,2012.000000,2.031470e+05,77.750000,79.000000,212.900000,82.750000,104.000000,6.700000,8.750000,9.500000,29.000000,34.500000,11.320000,3.100000,3.270000,10.000000
75%,2018.000000,1.629014e+06,80.000000,81.250000,233.000000,85.250000,107.000000,8.600000,9.000000,10.000000,31.500000,37.000000,11.720000,3.230000,3.360000,14.000000
max,2023.000000,1.962937e+09,89.250000,91.000000,314.000000,98.250000,122.500000,21.000000,10.500000,12.000000,39.500000,45.500000,13.440000,3.760000,3.810000,26.000000


However, we know that the NBA combine is invite only, and many players who get drafted do not go to the combine. For that, we can look towards the entirety of NCAA D1 Men's Basketball data.

## 2.3 Loading and Preprocessing NCAA Data

And let's also take a look at our NCAA D1 player stats, which we scraped from barttorvik.com, a live NCAA D1 player stats tracker by season.

In [ ]:
file_path = '/content/drive/MyDrive/final_project_datasets/battorvikPlayerData.xlsx'
ncaa_df = pd.read_excel(file_path)
display(ncaa_df.head(10))

,Rk,Player,Class,Team,Conf,Min%,PRPG!,BPM,ORtg,Usg,...,DR,Ast,TO,Blk,Stl,FTR,2P,3P/100,3P,Year
0,1,Bennett Stirtz,Jr,Drake,MVC,98.8,6.4,10.0,126.4,26.1,...,14.0,34.0,13.1,1.0,3.3,38.6,0.545,7.7,0.396,2025
1,2,Bruce Thornton,Jr,Ohio St.,B10,88.4,6.3,8.7,130.0,22.0,...,10.0,25.1,10.8,0.4,1.8,41.8,0.547,7.0,0.424,2025
2,3,Ryan Kalkbrenner,Sr,Creighton,BE,83.1,6.1,11.1,129.2,22.3,...,18.4,10.0,11.2,7.3,0.9,38.6,0.706,3.0,0.344,2025
3,4,Eric Dixon,Sr,Villanova,BE,84.3,6.1,7.3,116.7,32.9,...,12.6,13.1,11.8,1.1,1.5,35.8,0.483,13.1,0.407,2025
4,5,Cooper Flagg,Fr,Duke,ACC,72.8,6.0,14.9,123.0,30.8,...,21.2,26.8,13.5,4.9,2.8,42.9,0.517,7.2,0.385,2025
5,6,Trey Kaufman-Renn,Jr,Purdue,B10,76.9,5.7,7.9,118.1,31.1,...,15.4,16.8,13.9,1.2,1.4,42.8,0.597,0.4,0.429,2025
6,7,Johni Broome,Sr,Auburn,SEC,71.4,5.7,12.9,118.5,30.6,...,26.0,19.5,9.9,7.5,1.8,39.0,0.559,4.8,0.278,2025
7,8,Braden Smith,Jr,Purdue,B10,92.6,5.7,9.4,116.1,26.6,...,13.5,44.1,18.6,0.7,3.5,20.3,0.469,9.9,0.381,2025
8,9,Kam Jones,Sr,Marquette,BE,83.9,5.7,9.3,118.1,29.2,...,13.6,38.1,11.1,0.9,2.4,16.0,0.586,10.6,0.311,2025
9,10,Tyson Degenhart,Sr,Boise St.,MWC,84.5,5.6,7.6,126.8,23.8,...,15.3,10.3,10.7,0.9,1.1,50.5,0.618,7.6,0.349,2025


In [ ]:
ncaa_df.dtypes

,0
Rk,int64
Player,object
Class,object
Team,object
Conf,object
Min%,float64
PRPG!,float64
BPM,float64
ORtg,float64
Usg,float64


In [ ]:
ncaa_df.describe()

,Rk,Min%,PRPG!,BPM,ORtg,Usg,eFG,TS,OR,DR,Ast,TO,Blk,Stl,FTR,2P,3P/100,3P,Year
count,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000,39154.000000
mean,1088.863973,63.886001,1.601908,0.508939,103.134344,20.155992,50.251591,53.593196,5.069479,13.179956,13.647076,18.396751,1.797867,1.892037,35.759151,0.484399,5.994695,0.294807,2016.615569
std,629.257082,13.680132,1.343539,3.643385,10.595077,4.613991,6.271648,5.656281,3.587501,4.846323,7.603259,5.055464,2.141272,0.801671,15.741384,0.077856,4.120459,0.137440,5.194556
min,1.000000,40.000000,-2.700000,-13.300000,55.400000,6.100000,18.300000,20.300000,0.000000,2.100000,0.000000,2.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2008.000000
25%,544.000000,52.400000,0.700000,-2.000000,96.300000,16.800000,46.100000,49.900000,2.100000,9.500000,7.900000,14.900000,0.400000,1.300000,24.500000,0.433000,2.500000,0.262000,2012.000000
50%,1088.000000,63.900000,1.500000,0.400000,103.300000,19.900000,50.000000,53.600000,4.000000,12.500000,12.000000,17.900000,1.000000,1.800000,33.600000,0.483000,6.000000,0.330000,2017.000000
75%,1632.000000,74.975000,2.400000,2.900000,110.200000,23.200000,54.300000,57.300000,7.500000,16.300000,18.000000,21.300000,2.400000,2.300000,44.900000,0.535000,9.100000,0.374000,2021.000000
max,2308.000000,98.800000,7.900000,18.700000,161.400000,38.500000,88.300000,80.400000,23.700000,39.200000,52.600000,54.100000,18.800000,7.600000,147.900000,1.000000,23.100000,1.000000,2025.000000


## 2.4 Loading and Preprocessing NBA Draft Data

Let's also take a look at our NBA Draft data, which we scraped from basketball-reference.com, a live NBA tracker

In [ ]:
file_path = '/content/drive/MyDrive/final_project_datasets/nba_draft_2000_2025_clean.csv'
draft_df = pd.read_csv(file_path)
display(draft_df.head(10))

,Rk,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,FT%,MP.1,PTS.1,TRB.1,AST.1,WS,WS/48,BPM,VORP,Year
0,1.0,1.0,NJN,Kenyon Martin,Cincinnati,15.0,757,23134,9325,5159,...,.629,30.6,12.3,6.8,1.9,48.0,.100,0.1,12.1,2000
1,2.0,2.0,VAN,Stromile Swift,LSU,9.0,547,10804,4582,2535,...,.699,19.8,8.4,4.6,0.5,21.3,.095,-1.6,1.1,2000
2,3.0,3.0,LAC,Darius Miles,NaN,7.0,446,11730,4507,2190,...,.590,26.3,10.1,4.9,1.9,9.5,.039,-1.0,3.0,2000
3,4.0,4.0,CHI,Marcus Fizer,Iowa State,6.0,289,6032,2782,1340,...,.691,20.9,9.6,4.6,1.2,2.7,.022,-3.7,-2.6,2000
4,5.0,5.0,ORL,Mike Miller,Florida,17.0,1032,27812,10973,4376,...,.769,26.9,10.6,4.2,2.6,60.7,.105,0.8,19.8,2000
5,6.0,6.0,ATL,DerMarr Johnson,Cincinnati,7.0,344,5930,2121,769,...,.789,17.2,6.2,2.2,0.9,6.4,.052,-1.6,0.6,2000
6,7.0,7.0,CHI,Chris Mihm,Texas,8.0,436,8758,3262,2302,...,.704,20.1,7.5,5.3,0.5,13.3,.073,-3.9,-4.3,2000
7,8.0,8.0,CLE,Jamal Crawford,Michigan,20.0,1327,38994,19419,2948,...,.862,29.4,14.6,2.2,3.4,60.7,.075,-0.1,18.4,2000
8,9.0,9.0,HOU,Joel Przybilla,Minnesota,13.0,592,11733,2293,3665,...,.557,19.8,3.9,6.2,0.4,23.0,.094,-1.7,0.8,2000
9,10.0,10.0,ORL,Keyon Dooling,Missouri,13.0,728,14134,5067,964,...,.799,19.4,7.0,1.3,2.2,18.5,.063,-2.0,-0.2,2000


In [ ]:
draft_df.dtypes

,0
Rk,float64
Pk,float64
Tm,object
Player,object
College,object
Yrs,object
G,object
MP,object
PTS,object
TRB,object


In [ ]:
draft_df.describe()

,Rk,Pk,Year
count,1545.000000,1545.000000,1578.000000
mean,30.277023,30.257605,2012.555767
std,17.217246,17.200612,7.479822
min,1.000000,1.000000,2000.000000
25%,15.000000,15.000000,2006.000000
50%,30.000000,30.000000,2013.000000
75%,45.000000,45.000000,2019.000000
max,60.000000,60.000000,2025.000000


Here, we can see that only 3 columns in draft_df are of dtype int. This means that we will have to convert the rest of the columns from dtype object to int

## 2.5 Loading and Preprocessing NBA Player Data

We will also load in our NBA player data, which we scraped from ESPN and also basketball-reference.com. We will merge these two dataframes together later in part 3.

In [ ]:
file_path = '/content/drive/MyDrive/final_project_datasets/nba_player_stats_2000_2023_fixed_years_multiTM_clean.csv'
nba_br_df = pd.read_csv(file_path)
display(nba_br_df.head(10))

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Year
0,1.0,Shaquille O'Neal,27.0,LAL,C,79.0,79.0,40.0,12.1,21.1,...,9.4,13.6,3.8,0.5,3.0,2.8,3.2,29.7,"MVP-1,DPOY-2,AS,NBA1,DEF2",2000
1,2.0,Allen Iverson,24.0,PHI,SG,70.0,70.0,40.8,10.4,24.8,...,2.8,3.8,4.7,2.1,0.1,3.3,2.3,28.4,"MVP-7,AS,NBA2",2000
2,3.0,Grant Hill,27.0,DET,SF,74.0,74.0,37.5,9.4,19.2,...,5.3,6.6,5.2,1.4,0.6,3.2,2.6,25.8,"MVP-8,AS,NBA2",2000
3,4.0,Vince Carter,23.0,TOR,SF,82.0,82.0,38.1,9.6,20.7,...,4.0,5.8,3.9,1.3,1.1,2.2,3.2,25.7,"MVP-10,AS,NBA3",2000
4,5.0,Karl Malone,36.0,UTA,PF,82.0,82.0,35.9,9.2,18.0,...,7.4,9.5,3.7,1.0,0.9,2.8,2.8,25.5,"MVP-4,AS,NBA2",2000
5,6.0,Chris Webber,26.0,SAC,PF,75.0,75.0,38.4,10.0,20.6,...,8.0,10.5,4.6,1.6,1.7,2.9,3.5,24.5,"MVP-9,AS,NBA3",2000
6,7.0,Gary Payton,31.0,SEA,PG,82.0,82.0,41.8,9.1,20.3,...,5.2,6.5,8.9,1.9,0.2,2.7,2.2,24.2,"MVP-6,DPOY-5,AS,NBA1,DEF1",2000
7,8.0,Jerry Stackhouse,25.0,DET,SG,82.0,82.0,38.4,7.5,17.6,...,2.4,3.8,4.5,1.3,0.4,3.8,2.3,23.6,AS,2000
8,9.0,Tim Duncan,23.0,SAS,PF,74.0,74.0,38.9,8.5,17.3,...,8.9,12.4,3.2,0.9,2.2,3.3,2.8,23.2,"MVP-5,AS,NBA1,DEF1",2000
9,10.0,Kevin Garnett,23.0,MIN,PF,81.0,81.0,40.0,9.4,18.8,...,9.0,11.8,5.0,1.5,1.6,3.3,2.5,22.9,"MVP-2,DPOY-7,AS,NBA1,DEF1",2000


In [ ]:
nba_br_df.dtypes

,0
Rk,float64
Player,object
Age,float64
Team,object
Pos,object
G,float64
GS,float64
MP,float64
FG,float64
FGA,float64


In [ ]:
nba_br_df.describe()

,Rk,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
count,9836.000000,9836.000000,9836.000000,9836.000000,9836.000000,9836.000000,9836.000000,9833.000000,9836.000000,9836.000000,...,9836.000000,9836.000000,9836.000000,9836.000000,9836.000000,9836.000000,9836.000000,9836.000000,9836.000000,9838.000000
mean,203.451708,26.649654,56.489020,29.003863,22.458184,3.506415,7.760543,0.448383,0.727847,2.054036,...,1.022773,2.929504,3.950356,2.068625,0.705277,0.450397,1.294490,1.973546,9.407107,2011.359016
std,117.027192,4.274809,22.565843,29.042118,8.887688,2.112659,4.472559,0.074341,0.758573,1.992270,...,0.822219,1.772717,2.443491,1.839382,0.422173,0.480604,0.776622,0.726336,5.845014,6.941689
min,1.000000,18.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2000.000000
25%,103.000000,23.000000,42.000000,2.000000,15.400000,1.900000,4.300000,0.408000,0.000000,0.200000,...,0.400000,1.700000,2.200000,0.800000,0.400000,0.100000,0.700000,1.500000,5.000000,2005.000000
50%,204.000000,26.000000,63.000000,17.000000,22.100000,3.000000,6.700000,0.442000,0.500000,1.700000,...,0.800000,2.500000,3.400000,1.500000,0.600000,0.300000,1.100000,2.000000,7.900000,2011.000000
75%,303.000000,30.000000,75.000000,57.000000,29.800000,4.700000,10.400000,0.484000,1.200000,3.300000,...,1.400000,3.800000,5.100000,2.800000,0.900000,0.600000,1.700000,2.500000,12.500000,2017.000000
max,444.000000,44.000000,85.000000,83.000000,43.700000,12.200000,27.800000,1.000000,5.300000,13.200000,...,5.500000,11.500000,16.300000,11.700000,2.900000,5.000000,5.700000,6.000000,36.100000,2023.000000


In [ ]:
file_path = '/content/drive/MyDrive/final_project_datasets/nbaPlayerData.xlsx'
nba_espn_df = pd.read_excel(file_path)
display(nba_espn_df.head(10))

,RK,Player,Team,3P%,3PA,3PM,AST,BLK,DD2,FG%,...,FTM,GP,MIN,POS,PTS,REB,STL,TD3,TO,Year
0,-,Allen Iverson,PHI,29.1,4.5,1.3,5.5,0.2,4,39.8,...,7.9,60,43.7,SG,31.4,4.5,2.8,1,4.0,2002
1,-,Shaquille O'Neal,LAL,0.0,0.0,0.0,3.0,2.0,40,57.9,...,5.9,67,36.1,C,27.2,10.7,0.6,0,2.6,2002
2,-,Paul Pierce,BOS,40.4,6.3,2.6,3.2,1.0,17,44.2,...,6.3,82,40.3,SF,26.1,6.9,1.9,0,2.9,2002
3,-,Tracy McGrady,ORL,36.4,3.7,1.4,5.3,1.0,24,45.1,...,5.5,76,38.3,SG,25.6,7.9,1.6,1,2.5,2002
4,-,Tim Duncan,SA,10.0,0.1,0.0,3.7,2.5,67,50.8,...,6.8,82,40.6,C,25.5,12.7,0.7,0,3.2,2002
5,-,Kobe Bryant,LAL,25.0,1.7,0.4,5.5,0.4,11,46.9,...,6.1,80,38.3,SF,25.2,5.5,1.5,1,2.8,2002
6,-,Vince Carter,TOR,38.7,5.2,2.0,4.0,0.7,5,42.8,...,4.1,60,39.8,G,24.7,5.2,1.6,0,2.6,2002
7,-,Chris Webber,SAC,26.3,0.4,0.1,4.8,1.4,31,49.5,...,4.7,54,38.4,C,24.5,10.1,1.7,0,2.9,2002
8,-,Dirk Nowitzki,DAL,39.7,4.6,1.8,2.4,1.0,38,47.7,...,5.8,76,38.0,F,23.4,9.9,1.1,0,1.9,2002
9,-,Michael Jordan,WSH,18.9,0.9,0.2,5.2,0.4,8,41.6,...,4.4,60,34.9,G,22.9,5.7,1.4,0,2.7,2002


In [ ]:
nba_espn_df.dtypes

,0
RK,object
Player,object
Team,object
3P%,float64
3PA,float64
3PM,float64
AST,float64
BLK,float64
DD2,int64
FG%,float64


In [ ]:
nba_espn_df.describe()

,3P%,3PA,3PM,AST,BLK,DD2,FG%,FGA,FGM,FT%,FTA,FTM,GP,MIN,PTS,REB,STL,TD3,TO,Year
count,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000,11095.000000
mean,26.352591,2.040297,0.718558,1.907959,0.417071,4.434971,44.596413,7.078260,3.204380,71.484308,1.968247,1.489797,52.390626,20.571104,8.614430,3.648481,0.646868,0.151510,1.166886,2014.087787
std,16.682662,2.023629,0.772698,1.819998,0.453910,9.385283,8.959944,4.673945,2.216838,18.393246,1.758923,1.412914,24.372468,9.720884,6.121573,2.456222,0.427746,1.233593,0.798064,7.110542
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.100000,0.000000,0.000000,0.000000,0.000000,2002.000000
25%,15.400000,0.200000,0.000000,0.600000,0.100000,0.000000,40.300000,3.400000,1.500000,66.200000,0.800000,0.500000,34.000000,12.600000,3.900000,1.900000,0.300000,0.000000,0.600000,2008.000000
50%,32.100000,1.500000,0.500000,1.300000,0.300000,0.000000,44.200000,6.000000,2.700000,75.400000,1.400000,1.000000,59.000000,20.200000,7.100000,3.100000,0.600000,0.000000,1.000000,2014.000000
75%,37.200000,3.300000,1.200000,2.500000,0.500000,4.000000,48.700000,9.800000,4.500000,82.200000,2.600000,2.000000,73.000000,28.650000,11.900000,4.800000,0.900000,0.000000,1.600000,2020.000000
max,100.000000,13.200000,5.300000,11.700000,3.800000,77.000000,100.000000,27.800000,12.200000,100.000000,12.300000,10.200000,85.000000,43.700000,36.100000,16.000000,3.000000,42.000000,5.700000,2025.000000


We want to normalize the team format across dataframes. In nba_espn_df, players who played on multiple teams (IND/CHI, for example) are represented with a '/'. We will keep this format.

In [ ]:
def replace_teams(team_str):
    if not isinstance(team_str, str):  # handle NaN or other non-string values
        return team_str
    teams = team_str.split('/')
    if len(teams) == 1:
        return team_str  # single team stays as is
    else:
        return f"{len(teams)}TM"

nba_espn_df['Team'] = nba_espn_df['Team'].apply(replace_teams)


In [ ]:
nba_espn_df.head(40)

,RK,Player,Team,3P%,3PA,3PM,AST,BLK,DD2,FG%,...,FTM,GP,MIN,POS,PTS,REB,STL,TD3,TO,Year
0,-,Allen Iverson,PHI,29.1,4.5,1.3,5.5,0.2,4,39.8,...,7.9,60,43.7,SG,31.4,4.5,2.8,1,4.0,2002
1,-,Shaquille O'Neal,LAL,0.0,0.0,0.0,3.0,2.0,40,57.9,...,5.9,67,36.1,C,27.2,10.7,0.6,0,2.6,2002
2,-,Paul Pierce,BOS,40.4,6.3,2.6,3.2,1.0,17,44.2,...,6.3,82,40.3,SF,26.1,6.9,1.9,0,2.9,2002
3,-,Tracy McGrady,ORL,36.4,3.7,1.4,5.3,1.0,24,45.1,...,5.5,76,38.3,SG,25.6,7.9,1.6,1,2.5,2002
4,-,Tim Duncan,SA,10.0,0.1,0.0,3.7,2.5,67,50.8,...,6.8,82,40.6,C,25.5,12.7,0.7,0,3.2,2002
5,-,Kobe Bryant,LAL,25.0,1.7,0.4,5.5,0.4,11,46.9,...,6.1,80,38.3,SF,25.2,5.5,1.5,1,2.8,2002
6,-,Vince Carter,TOR,38.7,5.2,2.0,4.0,0.7,5,42.8,...,4.1,60,39.8,G,24.7,5.2,1.6,0,2.6,2002
7,-,Chris Webber,SAC,26.3,0.4,0.1,4.8,1.4,31,49.5,...,4.7,54,38.4,C,24.5,10.1,1.7,0,2.9,2002
8,-,Dirk Nowitzki,DAL,39.7,4.6,1.8,2.4,1.0,38,47.7,...,5.8,76,38.0,F,23.4,9.9,1.1,0,1.9,2002
9,-,Michael Jordan,WSH,18.9,0.9,0.2,5.2,0.4,8,41.6,...,4.4,60,34.9,G,22.9,5.7,1.4,0,2.7,2002


## 2.6 Loading and Preprocessing Advanced Statistics

Finally, we will load in our data of advanced statistics, which we also scraped from basketball-reference.com. We hope that these advanced statistics can include specific statistics that will make underdog or outperforming players stand out.

In [ ]:
file_path = '/content/drive/MyDrive/final_project_datasets/NBA_Advanced_stats_2008-2025_fixed_years_multiTM_clean.csv'
advanced_df = pd.read_csv(file_path)
display(advanced_df.head(10))

,Rk,Player,Age,Team,Pos,G,GS,MP,PER,TS%,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Awards,Year
0,1,Allen Iverson,32,DEN,SG,82,82,3424,20.9,0.567,...,8.9,2.8,11.6,0.163,3.4,-0.7,2.7,4.0,AS,2008
1,2,Joe Johnson,26,ATL,SG,82,82,3343,17.3,0.535,...,5.3,1.7,7.0,0.100,2.7,-1.5,1.2,2.7,AS,2008
2,3,Andre Iguodala,24,PHI,SF,82,82,3242,19.0,0.543,...,5.3,4.4,9.6,0.143,2.5,1.5,4.0,4.9,NaN,2008
3,4,Richard Jefferson,27,NJN,SF,82,82,3200,17.4,0.571,...,6.1,1.5,7.6,0.114,1.6,-1.3,0.3,1.8,NaN,2008
4,5,Baron Davis,28,GSW,PG,82,82,3196,19.8,0.523,...,5.9,2.8,8.7,0.131,3.2,0.3,3.6,4.5,NaN,2008
5,6,Kobe Bryant,29,LAL,SG,82,82,3192,24.2,0.576,...,9.5,4.3,13.8,0.208,5.2,0.6,5.8,6.3,"MVP-1,DPOY-5,AS,NBA1,DEF1",2008
6,7,Jamal Crawford,27,NYK,SG,80,80,3190,16.0,0.528,...,4.6,0.4,5.0,0.076,2.1,-1.8,0.3,1.9,NaN,2008
7,8,Jason Richardson,27,CHA,SF,82,82,3149,18.4,0.554,...,4.8,2.6,7.4,0.113,3.3,-0.3,3.0,4.0,NaN,2008
8,9,Dwight Howard,22,ORL,C,82,82,3088,22.9,0.619,...,6.4,6.4,12.9,0.200,1.6,1.2,2.8,3.7,"MVP-5,DPOY-7,AS,NBA1,DEF2",2008
9,10,Rashard Lewis,28,ORL,PF,81,81,3076,16.7,0.591,...,6.1,3.7,9.8,0.153,2.7,0.4,3.1,4.0,NaN,2008


In [ ]:
advanced_df.dtypes

,0
Rk,int64
Player,object
Age,Int64
Team,object
Pos,object
...,...
eFG,float64
hand_width,float64
PTS,float64
hand_length,float64


In [ ]:
advanced_df.describe()

,Rk,Age,GP,GS,MIN,PER,TS%,3PAr,FTr,ORB%,...,last_name,spot_college_corner_left,body_fat_pct,on_move_college,2P%,Usg,eFG,hand_width,PTS,hand_length
count,6557.000000,6557.0,6557.0,6557.0,6557.000000,6556.000000,6554.000000,6553.000000,6553.000000,6556.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,201.034009,26.52951,58.337959,29.256367,1394.789233,13.854866,0.539528,0.294548,0.271633,5.193975,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,114.557994,4.22162,18.852139,28.210774,757.288966,4.686067,0.060540,0.216604,0.145375,4.056865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,19.0,1.0,0.0,0.000000,-13.400000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,102.000000,23.0,46.0,3.0,755.000000,10.800000,0.506000,0.074000,0.172000,2.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,202.000000,26.0,63.0,18.0,1360.000000,13.400000,0.541000,0.306000,0.245000,3.700000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,300.000000,29.0,74.0,56.0,2003.000000,16.500000,0.576000,0.453000,0.339000,7.700000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,435.000000,43.0,83.0,83.0,3424.000000,76.300000,1.000000,1.000000,2.167000,63.900000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.7 Loading and Preprocessing NBA Draft Data

Finally, we need to load the NBA Draft data, which will assist in our EDA and modeling later.

In [ ]:
file_path = '/content/drive/MyDrive/final_project_datasets/nba_draft_2000_2025_clean.csv'
draft_df = pd.read_csv(file_path)
display(draft_df.head(10))

,Rk,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,FT%,MP.1,PTS.1,TRB.1,AST.1,WS,WS/48,BPM,VORP,Year
0,1.0,1.0,NJN,Kenyon Martin,Cincinnati,15.0,757,23134,9325,5159,...,.629,30.6,12.3,6.8,1.9,48.0,.100,0.1,12.1,2000
1,2.0,2.0,VAN,Stromile Swift,LSU,9.0,547,10804,4582,2535,...,.699,19.8,8.4,4.6,0.5,21.3,.095,-1.6,1.1,2000
2,3.0,3.0,LAC,Darius Miles,NaN,7.0,446,11730,4507,2190,...,.590,26.3,10.1,4.9,1.9,9.5,.039,-1.0,3.0,2000
3,4.0,4.0,CHI,Marcus Fizer,Iowa State,6.0,289,6032,2782,1340,...,.691,20.9,9.6,4.6,1.2,2.7,.022,-3.7,-2.6,2000
4,5.0,5.0,ORL,Mike Miller,Florida,17.0,1032,27812,10973,4376,...,.769,26.9,10.6,4.2,2.6,60.7,.105,0.8,19.8,2000
5,6.0,6.0,ATL,DerMarr Johnson,Cincinnati,7.0,344,5930,2121,769,...,.789,17.2,6.2,2.2,0.9,6.4,.052,-1.6,0.6,2000
6,7.0,7.0,CHI,Chris Mihm,Texas,8.0,436,8758,3262,2302,...,.704,20.1,7.5,5.3,0.5,13.3,.073,-3.9,-4.3,2000
7,8.0,8.0,CLE,Jamal Crawford,Michigan,20.0,1327,38994,19419,2948,...,.862,29.4,14.6,2.2,3.4,60.7,.075,-0.1,18.4,2000
8,9.0,9.0,HOU,Joel Przybilla,Minnesota,13.0,592,11733,2293,3665,...,.557,19.8,3.9,6.2,0.4,23.0,.094,-1.7,0.8,2000
9,10.0,10.0,ORL,Keyon Dooling,Missouri,13.0,728,14134,5067,964,...,.799,19.4,7.0,1.3,2.2,18.5,.063,-2.0,-0.2,2000


In [ ]:
draft_df.dtypes

In [ ]:
draft_df.describe()

# Part 3: Wrangling

### 3.1 Data Filtering

We can see that although ncaa_df ranges from 2008 - 2025, combine_df only ranges from 2001 - 2023. This indicates that we will have to clean and format our dataframes to only include relevant years for processing.

First, let us only include the years where the combine and NCAA overlap.

In [ ]:
dfs = [combine_df, ncaa_df, nba_espn_df, nba_br_df, advanced_df]

#Find intersection year range
intersection_min = max(df["Year"].min() for df in dfs)
intersection_max = min(df["Year"].max() for df in dfs)

#Filter out each data frame
combine_df   = combine_df[(combine_df["Year"] >= intersection_min) & (combine_df["Year"] <= intersection_max)]
ncaa_df      = ncaa_df[(ncaa_df["Year"] >= intersection_min) & (ncaa_df["Year"] <= intersection_max)]
nba_espn_df  = nba_espn_df[(nba_espn_df["Year"] >= intersection_min) & (nba_espn_df["Year"] <= intersection_max)]
nba_br_df    = nba_br_df[(nba_br_df["Year"] >= intersection_min) & (nba_br_df["Year"] <= intersection_max)]
advanced_df  = advanced_df[(advanced_df["Year"] >= intersection_min) & (advanced_df["Year"] <= intersection_max)]

intersection_min, intersection_max




(2008, 2023)

## 3.2 Data Merging

In this section, we will look to merge our data sets for EDA.

### 3.2.1 Normalizing Column Names

Next, we want to normalize our columns and dtypes across dataframes so that we are able to merge correctly.

In [ ]:
duplicate_groups = {
    'Player': ['PLAYER', 'player', 'PlayerName'],
    'Pos': ['Pos_br', 'POS_espn', 'Position'],
    'Age': ['Age_br', 'AGE'],
    'Height': ['HEIGHT', 'Height_br'],
    'Weight': ['WEIGHT', 'Weight_br'],
    'Year': ['Year_br', 'Year_espn', 'season', 'Season'],
    'Team': ['TEAM', 'Tm'],

    # Shooting + scoring
    'PTS': ['PTS_br', 'PTS_espn', 'Points'],
    'FGM': ['FGM_espn', 'FGM'],
    'FGA': ['FGA_br', 'FGA_espn'],
    'FG%': ['FG%_br', 'FG%_espn', 'FG_PCT'],
    '3PM': ['3PM', '3P'],
    '3PA': ['3PA_br', '3PA_espn'],
    '3P%': ['3P%_br', '3P%_espn', '3P_PCT'],
    'FTM': ['FTM_espn', 'FTM'],
    'FTA': ['FTA_br', 'FTA_espn'],
    'FT%': ['FT%_br', 'FT%_espn', 'FT_PCT'],

    # Rebounding + playmaking
    'TRB': ['TRB_br', 'REB', 'Rebounds', 'TRB_espn'],
    'AST': ['AST_br', 'AST_espn', 'Assists'],
    'STL': ['STL_br', 'STL_espn', 'Steals'],
    'BLK': ['BLK_br', 'BLK_espn', 'Blocks'],
    'TOV': ['TOV', 'TO', 'Turnovers'],

    # Games + minutes
    'GP': ['G', 'GP', 'Games'],
    'GS': ['GS', 'GamesStarted'],
    'MIN': ['MP', 'MIN', 'Minutes'],

    # Advanced stats
    'PER': ['PER_br', 'PER_espn'],
    'TS%': ['TS%', 'TS_PCT'],
    'USG%': ['USG%', 'UsageRate'],
    'WS': ['WS_br', 'WinShares'],
    'BPM': ['BPM_br', 'BoxPlusMinus'],
    'VORP': ['VORP_br']
}

In [ ]:
dfs = [combine_df, ncaa_df, nba_espn_df, nba_br_df, advanced_df]

# Apply renaming
for df in dfs:
    for unified_col, variants in duplicate_groups.items():
        for col in variants:
            if col in df.columns:
                df.rename(columns={col: unified_col}, inplace=True)

In [ ]:
# Determine the union of all columns after renaming
all_columns = set().union(*[df.columns for df in dfs])

# Add missing columns as NaN
for df in dfs:
    for col in all_columns:
        if col not in df.columns:
            df[col] = np.nan

# Reorder columns identically
for df in dfs:
    df = df[list(all_columns)]

### 3.2.2 Normalizing Column Datatypes

In [ ]:
# Columns that must be strings
string_cols = ['Player', 'Pos', 'Team']

# Columns that must be integers (or nullable ints)
int_cols = ['Age', 'Year', 'GP', 'GS']

# Columns that must be floats (percentages, rates, stats)
float_cols = [
    'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%',
    'TRB', 'AST', 'STL', 'BLK', 'TOV', 'MIN',
    'PER', 'TS%', 'USG%', 'WS', 'BPM', 'VORP',
    'Height', 'Weight'
]

# Apply dtype normalization across ALL dfs
for df in dfs:

    # --- Standardize string columns ---
    for col in string_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()

    # --- Standardize integer columns (nullable Int64 for missing values) ---
    for col in int_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

    # --- Standardize float columns ---
    for col in float_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype(float)


Next, we also know that there are players exist multiple times in each row. However, we want to take a look at their playing over time. We can differentiate these players by combining their name, team, and season for both NBA and NCAA data. This will be our merge key.

In [ ]:
for df in [combine_df, ncaa_df, nba_espn_df, nba_br_df, advanced_df]:
    # Remove whitespace, lowercase, normalize name
    df['Player'] = df['Player'].str.strip().str.lower()
    # Ensure 'Team' column is also standardized if it exists
    if 'Team' in df.columns:
        df['Team'] = df['Team'].astype(str).str.strip().str.lower()

    # Construct unique player_season_id (Player_Year) for all dataframes
    df['player_season_id'] = df['Player'] + "_" + df['Year'].astype(str)

    # Also construct player_team_season_id (Player_Team_Year) if 'Team' column exists
    if 'Team' in df.columns:
        df['player_team_season_id'] = df['Player'] + "_" + df['Team'] + "_" + df['Year'].astype(str)

/tmp/ipython-input-1189950380.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['player_season_id'] = df['Player'] + "_" + df['Year'].astype(str)
/tmp/ipython-input-1189950380.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['player_season_id'] = df['Player'] + "_" + df['Year'].astype(str)
/tmp/ipython-input-1189950380.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.con

In [ ]:
# Check that this is right
nba_br_df.head()

,Rk,Player,Age,Team,Pos,GP,GS,MIN,FG,FGA,...,off_drib_college_top_key,last_name,spot_college_corner_left,body_fat_pct,on_move_college,Usg,eFG,hand_width,hand_length,player_season_id
3380,1.0,lebron james,23,CLE,SF,75,74,40.4,10.6,21.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lebron james_2008
3381,2.0,kobe bryant,29,LAL,SG,82,82,38.9,9.5,20.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kobe bryant_2008
3382,3.0,allen iverson,32,DEN,SG,82,82,41.8,8.7,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,allen iverson_2008
3383,4.0,carmelo anthony,23,DEN,SF,77,77,36.4,9.5,19.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,carmelo anthony_2008
3384,5.0,amar'e stoudemire,25,PHO,C,79,79,33.9,9.0,15.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,amar'e stoudemire_2008


## 3.3 Data Merge

Finally, we want to merge our datasets so we can perform meaningful EDA analysis

### 3.3.1 NBA Data Merge

First, we want to merge our data for our two separate NBA dataframes, nba_espn_df and nba_br_df into nba_df

### 3.3.2 NBA Advanced Metrics Merge



merge nba_df with advanced_df into total_nba_df

### 3.3.3 NBA Draft and Combine Merge

merge combine_df and draft_df

### 3.3.4 College to NBA Draft Merge

merge ncaa_df to draft_df

### 3.3.5 NBA Draft to NBA Merge

merge draft_df to total_nba_df

### 3.3.6 College to NBA Merge

merge ncaa_df and total_nba_df

###

# Part 4: Exploratory Data Analysis

After generating a summary of our data and examining the overall data distribution, we can begin to explore insights and relationships between features, which will ultimately help us determine a better drafting procedure. In this section, we explore how NCAA performance statistics relate to the physical and athletic measurements collected at the NBA Combine. By comparing metrics such as wingspan, vertical leap, sprint time, and shooting or efficiency statistics, we aim to uncover relationships that may help explain how a player’s physical tools translate into on-court impact. These comparisons are important because they give us insight into which measurable physical attributes are associated with effective college performance and ultimately, which features may be most predictive when constructing a draft projection model. The goal is not only to visualize trends, but also to identify feature pairs where physical measurements meaningfully correlate with skill, efficiency, or overall impact.

## 4.1 NBA vs. Draft EDA

Take players in nba_total_df and match to draft_df. compare player's NBA advanced stats to their draft number and league-wide average advanced stats. Identify specific players with above-average advanced metrics and see how their draft position compares

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(combine_df['wingspan'], ncaa_df['BPM'])
plt.xlabel("Wingspan (inches)")
plt.ylabel("BPM")
plt.title("Wingspan vs College BPM")
plt.show()


Explanation

Also compare how draft position correlates with NBA outcomes such as awards, longterm advanced stats (PER, BPM, WS). Identify common advanced metrics that successful players share

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(combine_df['wingspan'], ncaa_df['BPM'])
plt.xlabel("Wingspan (inches)")
plt.ylabel("BPM")
plt.title("Wingspan vs College BPM")
plt.show()


Explanation

## 4.2 NCAA vs. Draft EDA

Compare draft pick with college performance by merging draft_df and ncaa_df. Identify which college stats translate best into draft position

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(combine_df['max_vertical_leap'].dropna(), bins=20)
plt.xlabel("Max Vertical Leap (inches)")
plt.ylabel("Count")
plt.title("Distribution of Max Vertical Leap")
plt.show()


Explanation

## 4.3 College vs. Draft vs. NBA

Average stats by draft pick and compare these averages to average college stats by pick and average nba stats by pick.

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(ncaa_df['eFG'], combine_df['three_quarter_sprint'])
plt.xlabel("NCAA eFG%")
plt.ylabel("Three-Quarter Sprint (sec)")
plt.title("eFG% vs Sprint Time")
plt.show()


Explanation

# Part 5: Feature Engineering & Preprocessing (All data sets in one)

# Part 6: Modeling

## 6.1 Logistic Regression Modeling

We wanted to predict whether a NBA prospect becomes an above-average NBA player. Using features from college statistics and draft information, we will train a logistic regression model because it provides a simple and interpretable way, before we move onto more advanced models.

## 6.2 Random Forest / XGBoost Pearson Correlation

## 6.3 Heatmap + Feature Correlation Matrix

## 6.4 Predicting the 2026 NBA Draft Class

# Part 7: Conclusion

First, we merge mo's career avg with ethan's career avg and get advanced metrics.

Then that with bennett's career averages.

Second, look at those players and compare to mo's draft dataframe and compare player's advanced stats and average those stats and look at players with above average stats and compare to their draft number.

Then, compare the same player's college numbers to the draft.

Draft to NBA and compare which types of players are getting any awards, top 3. and look at advanced metrics which players have in common

And merge combine data and draft data but before we compare draft nba and draft to college.

For draft dataset, we have ranking of draft and we can average stats by pick number. so we can have average stats by draft pick. with advanced metrics. and also for undrafted guys. and compare that to college stats and nba stats.

and now we are looking for certain stats that stand out between players who go from college to draft to nba and have certain metrics that line up together. more than one stat